# Importação

In [1]:
import config
from utils.functions import spark, median, length, col
from extract import extract_database_kaggle

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/27 21:43:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# spark.stop()

In [3]:
transacoes = extract_database_kaggle(spark, 'ealtman2019/credit-card-transactions')

# Análise exploratória

In [4]:
print(f'A base de transações tem {transacoes.count()} linhas e tem a seguinte composição: ')
transacoes.show(10)

A base de transações tem 24415009 linhas e tem a seguinte composição: 
+----+----+----+-----+---+-----+-------+-----------------+--------------------+-------------+--------------+-------+----+-------+---------+
|User|Card|Year|Month|Day| Time| Amount|         Use Chip|       Merchant Name|Merchant City|Merchant State|    Zip| MCC|Errors?|Is Fraud?|
+----+----+----+-----+---+-----+-------+-----------------+--------------------+-------------+--------------+-------+----+-------+---------+
|   0|   0|2002|    9|  1|06:21|$134.09|Swipe Transaction| 3527213246127876953|     La Verne|            CA|91750.0|5300|   NULL|       No|
|   0|   0|2002|    9|  1|06:42| $38.48|Swipe Transaction| -727612092139916043|Monterey Park|            CA|91754.0|5411|   NULL|       No|
|   0|   0|2002|    9|  2|06:22|$120.34|Swipe Transaction| -727612092139916043|Monterey Park|            CA|91754.0|5411|   NULL|       No|
|   0|   0|2002|    9|  2|17:45|$128.95|Swipe Transaction| 3414527459579106770|Monterey P

In [5]:
transacoes.createGlobalTempView("transacoes")

In [6]:
query = """--sql
    SELECT Year, count(*) as qtd_linhas 
    FROM global_temp.transacoes
    GROUP BY Year
    ORDER BY count(*) ASC
"""

linhas_ano = spark.sql(query).show()


25/03/27 21:43:46 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Card Brand
 Schema: Year
Expected: Year but found: Card Brand
CSV file: file:///Users/marcusferreira/.cache/kagglehub/datasets/ealtman2019/credit-card-transactions/versions/8/sd254_cards.csv
25/03/27 21:43:46 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Retirement Age
 Schema: Year
Expected: Year but found: Retirement Age
CSV file: file:///Users/marcusferreira/.cache/kagglehub/datasets/ealtman2019/credit-card-transactions/versions/8/sd254_users.csv


+----+----------+
|Year|qtd_linhas|
+----+----------+
|  52|         1|
|  77|         1|
|  79|         1|
|  76|         1|
|  78|         2|
|  50|         2|
|  53|         3|
|  54|         5|
|  56|         6|
|  55|         8|
|  75|        14|
|  57|        19|
|  74|        19|
|  58|        20|
|  59|        27|
|  73|        33|
|  60|        48|
|  72|        55|
|  61|        56|
|  63|        73|
+----+----------+
only showing top 20 rows



# Limpeza e filtros

In [7]:
transacoes_filtradas = transacoes.filter((length(col("Year")) == 4) & (col("Year").rlike("^[0-9]{4}$")))
transacoes_filtradas.createGlobalTempView("transacoes_filtradas")

In [8]:
query = """--sql
    SELECT DISTINCT Year
    FROM global_temp.transacoes_filtradas
"""

spark.sql(query).show(20) 

25/03/27 21:44:06 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Card Brand
 Schema: Year
Expected: Year but found: Card Brand
CSV file: file:///Users/marcusferreira/.cache/kagglehub/datasets/ealtman2019/credit-card-transactions/versions/8/sd254_cards.csv
25/03/27 21:44:06 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Retirement Age
 Schema: Year
Expected: Year but found: Retirement Age
CSV file: file:///Users/marcusferreira/.cache/kagglehub/datasets/ealtman2019/credit-card-transactions/versions/8/sd254_users.csv


+----+
|Year|
+----+
|2016|
|2012|
|2020|
|2019|
|2017|
|2014|
|2013|
|2005|
|2000|
|2002|
|2009|
|2018|
|2006|
|2004|
|2011|
|2008|
|1999|
|1997|
|2007|
|1996|
+----+
only showing top 20 rows

